In [1]:
#!/usr/bin/env python
# coding: utf-8

# In[ ]:


import torch 
import numpy as np
from torch.autograd import Variable
import time
import shutil
import os
from datetime import datetime
currentDateTime = datetime.now()
print("Date of Today : ", currentDateTime.month, " /", currentDateTime.day, "\nHour : ", currentDateTime.hour) 

#Date of Today
ctime = f"{currentDateTime.month}_{currentDateTime.day}_{currentDateTime.hour}h"
#torch.autograd.detect_anomaly(check_nan=True)

#Call model of layers and its forward step
from Forward_with_Layer_Setting import Net

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

#Call training functions of Loss functions
from NSpde_loss import lossNSpde
from BoundaryLoss import lossBdry
from InitialConditionLoss import lossIC
from Energy_Loss import Energy_loss
from Mass_Loss import Mass_loss


def create_network(load, loadfile):
    
    net = Net()
    net = net.to(device)


    #attempt to initial parameters if they exists
    if load == True:
        try:
            net.load_state_dict(torch.load(loadfile, map_location=torch.device(device)))
        except:
            print("\nLoading file was failed\n") #pass
        else:
            print("\nLoading file was completed\n")
    
    global epsilon #used to track loss
    epsilon = []
    global a_set #used to track a
    a_set = []
    global b_set #used to track b
    b_set = []
    
    a = 100
    b = 100
    
    print('Training PDE')
    start = time.time() #begin tracking computational time
    
    computation_time_vec = [0, 0, 0, 0] #used to record training time
    for i in range(4):
        #Set loop to optimize in progressively smaller learning rates
        if i == 0:
            #First loop uses progressively increasing time intervals
            print('\n\nExecuting Pass 1\n')
            time_slices = np.array([ 1])# .1,  .3, .6,
            iterations = 30000
            learning_rate = 10**-3    
        elif i == 1:
            print('\n\nExecuting Pass 2\n')
            time_slices = [time_slices[-1]]
            #time_slices = np.array([10])
            iterations = 30000
            learning_rate = 10**-3
        elif i == 2:
            print('\n\nExecuting Pass 3\n')
            time_slices = [time_slices[-1]]
            iterations = 30000
            learning_rate = 10**-4
        elif i ==3:
            print('\n\nExecuting Pass 4\n')
            iterations = 30000
            learning_rate = 10**-4
        
        training_loop(net, time_slices, iterations, learning_rate, IC_coefficient = 1, record_loss = 100, print_loss = 500, a = a, b = b)
        
        
        torch.save(net.state_dict(), f"{ctime}NNlayers_NSCH_{i}.pt")
        np.savetxt('epsilon.txt', epsilon)
        np.savetxt('a_set.txt', a_set)
        np.savetxt('b_set.txt', b_set)
        computation_time_vec[i] = time.time()

    np.savetxt('epsilon.txt', epsilon)

    print("Total Time:\t", computation_time_vec[-1]-start, '\nPass 1 Time:\t', computation_time_vec[0]-start, '\nPass 2 Time:\t', computation_time_vec[1]-computation_time_vec[0], '\nPass 3 Time:\t', computation_time_vec[2]-computation_time_vec[1], '\nPass 4 Time:\t', computation_time_vec[3]-computation_time_vec[2])
'''
def exponential_time_sample(collocation, t_l, t_u, r):
    t_pre = np.random.uniform(1,np.exp(10/4), size=(collocation, 1))
    
    #t = -np.log(1-t_pre+t_pre*np.exp(-r*(t_u-t_l)))/r
    #10/5 t = 5*np.tan(np.pi*t_pre/2)+5 ; -0.5,0.5,
    t = 4 * np.log(t_pre)
    return t 

def pdf_cust(net, x):
    # (a)
    # V = 1.5
    # pdf = 4/(x * np.sqrt(V*2*np.pi))*np.exp(-(np.log(x/4))**2/(2*V))
    
    #min_cust, max_cust = 0, 0.69522
    
    # (b)
    # x = torch.tensor(x).detach()
    # x = torch.reshape(x, (1, 1)).detach()
    # mass = net.mass(x)
    # mass = mass.detach().numpy()
    
    # pdf = np.exp(5*np.absolute(mass))
    
    # (c) not recommend
    # pdf = 1/(1+np.exp(-20 * x+10))+.03
    #max_cust = 1.03 #3.4038987  # maximum of pdf by pdf at pt file points at beginning
    
    # (d)
    V = 1.5
    pdf = 20/(x * np.sqrt(V*2*np.pi))*np.exp(-(np.log(x/20))**2/(2*2*V))
    
    min_cust, max_cust = 0, 1.459849
    return (pdf)/(max_cust)

def random_pdf_cust(net, collocation, t_l, t_u):
    t_pts=np.array([])
    
    while len(t_pts)< collocation:
        t =np.random.uniform(low=t_l+.000001,high=t_u) #
        
        density = pdf_cust(net, t)
        u = np.random.uniform(low=0,high=1)
        #print(density)
        #print(u)
        assert density>=0 and density<=1
        if u <=density:
            t_pts = np.append(t_pts, t)
            #print(t_pts)
    t_pts = np.resize(t_pts, (collocation,1))
    return t_pts
'''

def SpatialTrainPts(net, Domain_collocation, Bdry_collocation, IC_collocation, t_l, t_f): # Mass_collocation, Energy_collocation,
    #Set of all the recorded xy variables as base data for chasing during training
    
    # Domain boundary in the range [0, 1]x[0, 2] and time in [0, 1].
    x_l = net.x_l
    x_u = net.x_u
    y_l = net.y_l
    y_u = net.y_u
    z_l = net.z_l
    z_u = net.z_u

    #Pick NSpde Condition Training Random Points in Numpy
    x_domain = np.random.uniform(low= x_l, high=x_u, size=(Domain_collocation, 1))
    #x_domain = np.random.uniform(low= -1, high=1, size=(Domain_collocation, 1))
    y_domain = np.random.uniform(low= y_l, high=y_u, size=(Domain_collocation, 1)) 
    #y_domain = np.random.uniform(low= -1, high=1, size=(Domain_collocation, 1)) 
    z_domain = np.random.uniform(low= z_l, high=z_u, size=(Domain_collocation, 1)) 
    
    #10.11
    t_domain = np.random.uniform(low= t_l, high=t_f, size=(Domain_collocation, 1)) 
    #t_domain = exponential_time_sample(Domain_collocation, t_l, t_f, 0.2)
    
    
    #10.9 t_domain = random_pdf_cust(Domain_collocation, t_l, t_f)
    # t_domain = random_pdf_cust(net, Domain_collocation, t_l, t_f)
    
    #x_domain = (x_domain**3+1)/2
    #y_domain = (y_domain**3+1)/2
    
    #Move to pytorch tensors
    x_domain = Variable(torch.from_numpy(x_domain).float(), requires_grad=True).to(device)
    y_domain = Variable(torch.from_numpy(y_domain).float(), requires_grad=True).to(device)
    z_domain = Variable(torch.from_numpy(z_domain).float(), requires_grad=True).to(device)

    t_domain = Variable(torch.from_numpy(t_domain).float(), requires_grad=True).to(device)
    
    
    #Pick IC Training t starting points to make tensor
    x_IC = np.random.uniform(low= x_l, high=x_u, size=(IC_collocation, 1)) 
    y_IC = np.random.uniform(low= y_l, high=y_u, size=(IC_collocation, 1)) 
    z_IC = np.random.uniform(low= z_l, high=z_u, size=(IC_collocation, 1)) 

    t_IC = np.random.uniform(low= t_l, high=t_l, size=(IC_collocation, 1)) 
    
    '''
    x_IC = np.linspace( x_l, x_u, num=IC_collocation).reshape((IC_collocation, 1)) #linspace
    y_IC = np.linspace(y_l, y_u, num=IC_collocation).reshape((IC_collocation, 1))
    z_IC = np.linspace(z_l, z_u, num=IC_collocation).reshape((IC_collocation, 1))

    t_IC =  np.linspace(t_l, t_l, num=IC_collocation).reshape((IC_collocation, 1))
    '''
    x_IC = Variable(torch.from_numpy(x_IC).float(), requires_grad=True).to(device)
    y_IC = Variable(torch.from_numpy(y_IC).float(), requires_grad=True).to(device)
    z_IC = Variable(torch.from_numpy(z_IC).float(), requires_grad=True).to(device)

    t_IC = Variable(torch.from_numpy(t_IC).float(), requires_grad=True).to(device)
    
    #Pick BC Training Random Points in Numpy 
    x_Bdry = np.random.uniform(low=x_l, high=x_u, size=(Bdry_collocation,1))
    y_Bdry = np.random.uniform(low=y_l, high=y_u, size=(Bdry_collocation,1))
    z_Bdry = np.random.uniform(low=z_l, high=z_u, size=(Bdry_collocation,1))

    t_Bdry = np.random.uniform(low=t_l, high=t_f, size=(Bdry_collocation,1))      
    
    '''
    x_Bdry = np.linspace( x_l,  x_u,num=Bdry_collocation).reshape( (Bdry_collocation,1))
    y_Bdry = np.linspace( y_l, y_u, num=Bdry_collocation).reshape( (Bdry_collocation,1))
    z_Bdry = np.linspace( z_l,  z_u, num=Bdry_collocation).reshape( (Bdry_collocation,1))

    t_Bdry = np.linspace(t_l, t_f,num=Bdry_collocation).reshape( (Bdry_collocation,1))      
    '''
    #Move to pytorch tensors
    x_Bdry= Variable(torch.from_numpy(x_Bdry).float(), requires_grad=True).to(device)
    y_Bdry = Variable(torch.from_numpy(y_Bdry).float(), requires_grad=True).to(device)
    z_Bdry = Variable(torch.from_numpy(z_Bdry).float(), requires_grad=True).to(device)

    t_Bdry = Variable(torch.from_numpy(t_Bdry).float(), requires_grad=True).to(device)
         
    return x_domain, y_domain, z_domain, t_domain, x_IC, y_IC, z_IC, t_IC, x_Bdry, y_Bdry, z_Bdry, t_Bdry #, t_mass, t_energy

'''
def twoDimTrainPts_IC_Only(net, IC_collocation):
    #Set of all the recorded xy variables as base data for chasing during training
    
    # Domain boundary in the range [0, 1]x[0, 2] and time in [0, 1].
    x_l = net.x1_l
    x_u = net.x1_u
    y_l = net.x2_l
    y_u = net.x2_u
    
    #Pick IC Training t starting points to make tensor
    x_IC = np.linspace(low= x_l, high=x_u, size=(IC_collocation, 1)) 
    y_IC = np.random.uniform(low= y_l, high=y_u, size=(IC_collocation, 1)) 
    
    x_IC = Variable(torch.from_numpy(x_IC).float(), requires_grad=True).to(device)
    y_IC = Variable(torch.from_numpy(y_IC).float(), requires_grad=True).to(device)
    t_IC = Variable(torch.zeros_like(x_IC), requires_grad=True).to(device)
            
    return x_IC, y_IC, t_IC
'''    
def get_lr(optimizer):
    for param_group in optimizer.param_groups:
        return param_group['lr']
'''
def Create_IC_Parameters(Net, iterations, IC_collocation, learning_rate, filename, record_loss, print_loss):
    ICnet = Net.to(device)
    
    IC_loss = IC_Only_training(ICnet, iterations, IC_collocation, learning_rate, record_loss, print_loss)
    
    torch.save(ICnet.state_dict(), filename)
    
    return IC_loss

def IC_Only_training(net, iterations, IC_collocation, learning_rate, record_loss, print_loss):

    #learning rate update
    for g in net.optimizer.param_groups:
        g['lr'] = learning_rate
    
    #training loop
    epsilon_IC = [] #placeholder to track decreasing loss
    for epoch in range(1, iterations+1):
    
        # Resetting gradients to zero
        lbfgs.zero_grad() #net.optimizer.zero_grad()
           
        #get sample points
        x_IC, y_IC, t_IC = twoDimTrainPts_IC_Only(net, IC_collocation)
        
        #Loss based on Initial Condition
        loss = lossIC(net, x_IC, y_IC, t_IC)
           
        loss.backward()
        
        # Gradient Norm Clipping
        torch.nn.utils.clip_grad_norm_(net.parameters(), max_norm= 1, norm_type=2, error_if_nonfinite=True)

        #Gradient Value Clipping
        #nn.utils.clip_grad_value_(net.parameters(), clip_value=1.0)
        
        lbfgs.optimizer.step() #net.optimizer.step()
           
        #Print Loss every 1000 Epochs
        with torch.autograd.no_grad():
            
            if epoch%record_loss == 0:
                epsilon_IC = np.append(epsilon_IC, loss.cpu().detach().numpy())
            if epoch%print_loss == 0:
                print("Iteration:", epoch, "IC Loss:", loss.item())
    
    np.savetxt('epsilon_IC.txt', epsilon_IC)
    
    return loss.item()

'''
def training_loop(net, time_slices, iterations, learning_rate, IC_coefficient, record_loss, print_loss,a,b):
    #Main training loop function
    global epsilon #used to track and record loss
    global b_set
    global a_set
    #Set numbers of collocation points
    Domain_collocation = 10000 #3d sample points
    Bdry_collocation = 6*31**2 #2d sample points
    IC_collocation = 6*31**2 #2d sample points
    #Mass_collocation = 102
    #Energy_collocation = 103
    lamba = .1
    
    #update the learning rate as desired
    for g in net.optimizer.param_groups:
        g['lr'] = learning_rate
    
    #Iterate over time slices
    for final_time in time_slices:
        min_loss = 50
        min_mse_NS = 50
        min_mse_IC = 1
        
        
        #Print relevant hyperparameters (final sampling time and learning rate)
        with torch.autograd.no_grad():
            print("Current Final Time:", final_time, "Current Learning Rate: ", get_lr(net.optimizer))  
        
        #Set lower bound of time
        t_l = 0
        
        #Begin Iterative Procedure
        for epoch in range(1, iterations+1):
            
            #Choose Sample Points #, t_mass, t_energy
            x_domain, y_domain, z_domain, t_domain, x_IC, y_IC, z_IC, t_IC, x_Bdry, y_Bdry, z_Bdry, t_Bdry = SpatialTrainPts(net, Domain_collocation, Bdry_collocation, IC_collocation, t_l, final_time)
            
            # Resetting gradients to zero
            net.optimizer.zero_grad() #net.optimizer.zero_grad()
            '''
            ##################################################################################
            for param in net.parameters():
                print(param)
            '''
            #Loss based on Initial Condition
            mse_IC = lossIC(net, x_IC, y_IC, z_IC, t_IC)
            
            #Loss based on Boundary Condition (Containing No-Slip and Free-slip)
            boundary_domain_loss = lossBdry(net, x_Bdry, y_Bdry, z_Bdry, t_Bdry)
            mse_BC = boundary_domain_loss 
            
            #Loss based on PDE
            e_VP1, e_VP2, e_VP3, e_VP4 = lossNSpde(net, x_domain, y_domain, z_domain, t_domain)
            mse_NS = e_VP1 + e_VP2 + e_VP3 + e_VP4
            
            #Compute raw Loss (Used for our own reference)
            raw_loss = mse_IC + mse_BC + mse_NS  #+ mse_mass + mse_energy
            
            #Compute Weighted Loss (Used for iterative procedure)
            loss =   100*mse_BC + 100 * mse_IC + mse_NS #+ 1000*mse_phi #+ mse_mass + mse_energy
            '''
            print(net.parameters().is_leaf)

            ##############################################################################################
            #compute next step
            grad_pde_loss = torch.autograd.grad(mse_NS.sum(), param, create_graph=True, retain_graph=True)[0]
            grad_IC_loss = torch.autograd.grad(mse_IC.sum(), net.parameters(), create_graph=True, retain_graph=True)[0]
            grad_BC_loss = torch.autograd.grad(mse_BC.sum(), net.parameters(), create_graph=True, retain_graph=True)[0]
            # coeffiecient of BC => a, coeffiecient of IC => b
            a_next = (torch.max(torch.abs(grad_pde_loss))/(a*torch.max(torch.abs(grad_IC_loss)))).item() #10*
            b_next = (torch.max(torch.abs(grad_pde_loss))/(b*torch.max(torch.abs(grad_BC_loss)))).item() #10*
        
            # #update rates
            a = (1-lamba) * a + lamba * a_next
            b = (1-lamba) * b + lamba * b_next
            '''
            
            #step backward to compute gradients
            loss.backward()
            # Gradient Norm Clipping
            torch.nn.utils.clip_grad_norm_(net.parameters(), max_norm = .1, error_if_nonfinite=False) #note max norm is more art than science
            
            #Make Iterative Step
            
            def closure():
                return loss
            
            net.optimizer.step(closure) #net.optimizer.step(closure)
            
            #Print and Record Raw Loss # "\n\tMass Loss: ", mse_mass.item(), "\tEnergy Loss: ", mse_energy.item()
            with torch.autograd.no_grad():
                if epoch%record_loss == 0:
                    epsilon = np.append(epsilon, raw_loss.cpu().detach().numpy())
                    #a_set = np.append(a_set, a.cpu().detach().numpy())
                    #b_set = np.append(b_set, b.cpu().detach().numpy())
                if epoch%print_loss == 0:
                    u_s,v_s,w_s,P_s = net(x_domain, y_domain, z_domain, t_domain)
                    u_s_val, v_s_val, w_s_val, P_s_val = net.u_eq(x_domain, y_domain, z_domain, t_domain), net.v_eq(x_domain, y_domain, z_domain, t_domain), net.w_eq(x_domain, y_domain, z_domain, t_domain), net.P_eq(x_domain, y_domain, z_domain, t_domain)
                    
                    #u_b,v_b,w_b,P_b = net(x_Bdry, y_Bdry, z_Bdry, t_Bdry)
                    #u_b_val, v_b_val, w_b_val, P_b_val = net.u_eq(x_Bdry, y_Bdry, z_Bdry, t_Bdry), net.v_eq(x_Bdry, y_Bdry, z_Bdry, t_Bdry), net.w_eq(x_Bdry, y_Bdry, z_Bdry, t_Bdry), net.P_eq(x_Bdry, y_Bdry, z_Bdry, t_Bdry)
                    
                    #u_i,v_i,w_i,P_i = net(x_IC, y_IC, z_IC, t_IC)
                    #u_i_val, v_i_val, w_i_val, P_i_val = net.u_eq(x_IC, y_IC, z_IC, t_IC), net.v_eq(x_IC, y_IC, z_IC, t_IC), net.w_eq(x_IC, y_IC, z_IC, t_IC), net.P_eq(x_IC, y_IC, z_IC, t_IC)
                    
                    rel_L2_u = 1/(Domain_collocation ) *(torch.sum(((u_s - u_s_val)**2/u_s_val**2)**.5))
                    rel_L2_v = 1/(Domain_collocation ) *(torch.sum(((v_s - v_s_val)**2/v_s_val**2)**.5))
                    rel_L2_w = 1/(Domain_collocation ) *(torch.sum(((w_s - w_s_val)**2/w_s_val**2)**.5))
                    rel_L2_P = 1/(Domain_collocation ) *(torch.sum(((P_s - P_s_val)**2/P_s_val**2)**.5))
                    
                    print("Iteration:", epoch, "\tTotal Loss:", loss.data)
                    print("u relative L2 errors: ", rel_L2_u.item(), "\tv relative L2 errors: ", rel_L2_v.item(),
                          "\tw relative L2 errors: ", rel_L2_w.item(), "\tP relative L2 errors: ", rel_L2_P.item(),
                        "\nIC Loss: ", mse_IC.item(), "\tDirichlet Bdry Loss: ", boundary_domain_loss.item(), "\tNS PDE Loss: ", mse_NS.item(),
                        "\ne_VP1 Loss: ", e_VP1.item(), "\t\te_VP2 Loss: ", e_VP2.item(), "\t\te_VP3 Loss: ", e_VP3.item(), "\t\te_VP4 Loss: ", e_VP4.item(), )
                    if mse_NS.cpu().detach().numpy() < min_mse_NS:
                        min_mse_NS = mse_NS.cpu().detach().numpy()
                        torch.save(net.state_dict(), f"ES_min_NSloss_lr{get_lr(net.optimizer)}_t{final_time}_{ctime}.pt")
                        print('  *Saved ; Early Stopping for the min NS PDE Loss\n')
                    if mse_IC.cpu().detach().numpy() < min_mse_IC:
                        min_mse_IC = mse_IC.cpu().detach().numpy()
                        torch.save(net.state_dict(), f"ES_min_mse_IC_lr{get_lr(net.optimizer)}_t{final_time}_{ctime}.pt")
                        print('  *Saved ; Early Stopping for the min IC Loss\n')
                    if loss.cpu().detach().numpy() < min_loss:
                        min_loss = loss.cpu().detach().numpy()
                        torch.save(net.state_dict(), f"ES_min_loss_lr{get_lr(net.optimizer)}_t{final_time}_{ctime}.pt")
                        print('  *Saved ; Early Stopping for the Minimal Total Loss\n')
                            
                        
        
        if final_time != time_slices[-1]:      
            torch.save(net.state_dict(), f"{ctime}NNlayers_lr{get_lr(net.optimizer)}_Bubble_ft_{final_time}.pt")
                
#create_network(True)
create_network(load=True, loadfile = "2_2_10hNNlayers_NSCH_0.pt")


# In[ ]:

Date of Today :  2  / 2 
Hour :  17

Loading file was completed

Training PDE


Executing Pass 1

Current Final Time: 1 Current Learning Rate:  0.001
Iteration: 500 	Total Loss: tensor(0.0476)
u relative L2 errors:  0.008960502222180367 	v relative L2 errors:  0.007272059563547373 	w relative L2 errors:  0.03587711229920387 	P relative L2 errors:  0.06032602861523628 
IC Loss:  8.455105125904083e-05 	Dirichlet Bdry Loss:  0.0002671129477676004 	NS PDE Loss:  0.012397976592183113 
e_VP1 Loss:  0.0033760257065296173 		e_VP2 Loss:  0.003925060387700796 		e_VP3 Loss:  0.004992086905986071 		e_VP4 Loss:  0.00010480358469067141
  *Saved ; Early Stopping for the min NS PDE Loss

  *Saved ; Early Stopping for the min IC Loss

  *Saved ; Early Stopping for the Minimal Total Loss

Iteration: 1000 	Total Loss: tensor(0.0470)
u relative L2 errors:  0.00969809852540493 	v relative L2 errors:  0.007424218580126762 	w relative L2 errors:  0.058396466076374054 	P relative L2 errors:  0.065579973161220

KeyboardInterrupt: 